In [69]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [254]:
train = pd.read_csv(r"data/train.csv")
# train.drop("Store", axis=1, inplace=True)
# train.drop("Dept", axis=1, inplace=True)
# train.drop("Date", axis=1, inplace=True)
train.drop("MarkDown1", axis=1, inplace=True)
train.drop("MarkDown2", axis=1, inplace=True)
train.drop("MarkDown3", axis=1, inplace=True)
train.drop("MarkDown4", axis=1, inplace=True)
train.drop("MarkDown5", axis=1, inplace=True)
# test.drop("Id", axis = 1, inplace = True)
# g = sns.PairGrid(train)
# g.map(sns.scatterplot)
import calendar
dates = train["Date"]
def convertDate(date):
    # temp = date.split("-")
    # month = int(temp[1])
    # day = int(temp[2])
    # sum = 0
    # big = [1, 3, 5, 7, 8, 10, 12]
    # small = [4, 6, 9, 11]
    # for i in range(1, month):
    #     if i == 2:
    #         if calendar.isleap(int(temp[0])):
    #             sum += 29
    #         else:
    #             sum += 28
    #     elif i in big:
    #         sum += 31
    #     elif i in small:
    #         sum += 30
    #     else:
    #         raise "Unexpected I"
    # sum += day
    # return sum
    temp = date.split("-")
    return int(temp[1])

new_dates = []

for idx, date in enumerate(dates):
    new_dates.append(convertDate(date))
    # print(convertDate(date))

train["Date"] = new_dates
# train["Date"]
# train["IsHoliday"] = train["IsHoliday"].astype(int)
# train["Type"].replace(["A", "B", "C"], [0, 1, 2], inplace=True)

In [255]:
# train.hist(figsize=(20, 8))
# train["Unemployment"] = np.log(train["Unemployment"])
# train.hist(figsize=(20, 8))
train["Temperature"] = (train["Temperature"] - np.mean(train["Temperature"])) / np.std(train["Temperature"])
train["Temperature"] = np.sqrt(train["Temperature"])
train["Weekly_Sales"] = (train["Weekly_Sales"] - np.mean(train["Weekly_Sales"])) / np.std(train["Weekly_Sales"])
train["Fuel_Price"] = (train["Fuel_Price"] - np.mean(train["Fuel_Price"])) / np.std(train["Fuel_Price"])
train["Fuel_Price"] = np.sqrt(train["Fuel_Price"])
train["Unemployment"] = (train["Unemployment"] - np.mean(train["Unemployment"])) / np.std(train["Unemployment"])
train["Unemployment"] = np.log(train["Unemployment"])
train["Size"] = (train["Size"] - np.mean(train["Size"])) / np.std(train["Size"])
# train["Size"] = np.sqrt(train["Size"])

# train.hist(figsize=(20, 8))
one_hot = pd.get_dummies(train["Type"], prefix="Type")
train.drop("Type", axis=1, inplace=True)
train = train.join(one_hot)
one_hot = pd.get_dummies(train["IsHoliday"], prefix="IsHoliday")
train.drop("IsHoliday", axis=1, inplace=True)
train = train.join(one_hot)
one_hot = pd.get_dummies(train["Dept"], prefix="Dept")
train.drop("Dept", axis=1, inplace=True)
train = train.join(one_hot)
one_hot = pd.get_dummies(train["Store"], prefix="Store")
train.drop("Store", axis=1, inplace=True)
train = train.join(one_hot)

C:\Users\suinu\miniconda3\envs\walmart\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\suinu\miniconda3\envs\walmart\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [257]:

train["Temperature"] = train["Temperature"].fillna(np.mean(train["Temperature"]))
train["Fuel_Price"] = train["Fuel_Price"].fillna(np.mean(train["Fuel_Price"]))
train["Unemployment"] = train["Unemployment"].fillna(np.mean(train["Unemployment"]))
# train["Unemployment"]

In [259]:
y_train = train["Weekly_Sales"]
X_train = train.drop("Weekly_Sales", axis=1)
# train.isna().any()

Date            False
Weekly_Sales    False
Temperature     False
Fuel_Price      False
CPI             False
                ...  
Store_41        False
Store_42        False
Store_43        False
Store_44        False
Store_45        False
Length: 138, dtype: bool

In [260]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train)
    rmse = np.sqrt(-cross_val_score(model, X_train, y_train, scoring="neg_mean_squared_error", cv=kf))
    return(rmse)

lasso = make_pipeline(RobustScaler(), Lasso(alpha=0.0005, random_state=42))

score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Lasso score: 0.5865 (0.0119)



In [16]:
KRR = KernelRidge(alpha=1, kernel='linear')
score = rmsle_cv(KRR)
print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Kernel Ridge score: nan (nan)



C:\Users\suinu\miniconda3\envs\walmart\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\suinu\miniconda3\envs\walmart\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\suinu\miniconda3\envs\walmart\lib\site-packages\sklearn\kernel_ridge.py", line 197, in fit
    K = self._get_kernel(X)
  File "C:\Users\suinu\miniconda3\envs\walmart\lib\site-packages\sklearn\kernel_ridge.py", line 155, in _get_kernel
    return pairwise_kernels(X, Y,

In [20]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468,
                             learning_rate=0.05, max_depth=3,
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state=7, nthread=-1)
score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

[15:28:45] WARNING: ..\src\learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[15:29:40] WARNING: ..\src\learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[15:30:37] WARNING: ..\src\learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting fl

KeyboardInterrupt: 

In [44]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10,
                                   loss='huber', random_state =5)
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

KeyboardInterrupt: 

In [34]:
X_train = np.expand_dims(X_train, axis=2)
X_train = X_train.reshape((X_train.shape[0], 1, 7))

In [35]:
print(X_train.shape)

(282451, 1, 7)


In [41]:
import keras
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()

model.add(Dense(5, input_shape=(1, 7)))
# model.add(Dense(5))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
H = model.fit(X_train, y_train, epochs=50, batch_size=128)

Epoch 1/50
2207/2207 [==============================] - 1s 662us/step - loss: 485950432.0000
Epoch 2/50
2207/2207 [==============================] - 1s 645us/step - loss: 485227968.0000
Epoch 3/50
2207/2207 [==============================] - 1s 677us/step - loss: 485058272.0000
Epoch 4/50
 324/2207 [===>..........................] - ETA: 1s - loss: 486742048.0000

KeyboardInterrupt: 